# 01 — Dosimetry vs MDV Benchmark

Reproduce the UV dose calculations and NOAA MDV scaling used for the chamber experiment. Raw Solarmeter readings and schedules live in `data/raw/supplements/July_26_2025_DOSE_SCHEDULE_UPDATED/Calibration_files/`.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

PROJECT_ROOT = Path.cwd()
CALIB_DIR = PROJECT_ROOT / 'data/raw/supplements/July_26_2025_DOSE_SCHEDULE_UPDATED/Calibration_files'
OUTPUT_PATH = PROJECT_ROOT / 'data-processed/supplements/uv_dose_summary.csv'


In [ ]:
uva_raw = pd.read_csv(CALIB_DIR / 'UVA_Readings__Raw_.csv')
uvb_raw = pd.read_csv(CALIB_DIR / 'UVB_Readings__Raw_.csv')
print(uva_raw.head())
print(uvb_raw.head())


In [ ]:
# Average irradiance per grid
uva_power = uva_raw.groupby('grid')['irradiance_mW_cm2'].mean().rename('p_uva_mw_cm2')
uvb_power = uvb_raw.groupby('grid')['irradiance_mW_cm2'].mean().rename('p_uvb_mw_cm2')
chamber = pd.read_csv(CALIB_DIR / 'chamber_dose_schedule.csv')
chamber = chamber.rename(columns={
    'grid (#)': 'grid_id',
    'UVA_hours (h)': 'uva_hours',
    'UVB_hours (h)': 'uvb_hours',
    'P_UVA (mW cm^-2)': 'p_uva_schedule',
    'P_UVB (mW cm^-2)': 'p_uvb_schedule',
    '%MDV_UVA_dose (%)': 'mdv_uva_percent_schedule',
    '%MDV_UVB_dose (%)': 'mdv_uvb_percent_schedule',
})
doses = chamber.merge(uva_power, left_on='grid_id', right_index=True)
doses = doses.merge(uvb_power, left_on='grid_id', right_index=True)
doses.head()


In [ ]:
# Compute dose (mJ/cm^2 per day)
SECONDS_PER_HOUR = 3600

doses['uva_dose_mJ_cm2'] = doses['p_uva_mw_cm2'] * doses['uva_hours'] * SECONDS_PER_HOUR
doses['uvb_dose_mJ_cm2'] = doses['p_uvb_mw_cm2'] * doses['uvb_hours'] * SECONDS_PER_HOUR

# NOAA MDV reference powers
df_mdv = pd.read_csv(CALIB_DIR / 'MDV_24h_average_power.csv')
MDV_UVB = float(df_mdv['UVB_mW_cm2'].iloc[0])
MDV_UVA = float(df_mdv['UVA_mW_cm2'].iloc[0])

doses['mdv_uva_dose_mJ_cm2'] = MDV_UVA * 24 * SECONDS_PER_HOUR
uvb_ref_hours = doses['uvb_hours'].iloc[0]
doses['mdv_uvb_dose_mJ_cm2'] = MDV_UVB * uvb_ref_hours * SECONDS_PER_HOUR

doses['mdv_uva_percent_calc'] = 100 * doses['uva_dose_mJ_cm2'] / doses['mdv_uva_dose_mJ_cm2']
doses['mdv_uvb_percent_calc'] = 100 * doses['uvb_dose_mJ_cm2'] / doses['mdv_uvb_dose_mJ_cm2']
doses[['grid_id', 'p_uva_mw_cm2', 'p_uvb_mw_cm2', 'mdv_uva_percent_calc', 'mdv_uvb_percent_calc']].head()


In [ ]:
# Validate against provided schedule values
check = doses[['grid_id', 'mdv_uva_percent_calc', 'mdv_uvb_percent_calc', 'mdv_uva_percent_schedule', 'mdv_uvb_percent_schedule']]
check['delta_uva'] = check['mdv_uva_percent_calc'] - check['mdv_uva_percent_schedule']
check['delta_uvb'] = check['mdv_uvb_percent_calc'] - check['mdv_uvb_percent_schedule']
print(check)
assert (check['delta_uva'].abs() < 1e-3).all()
assert (check['delta_uvb'].abs() < 1e-3).all()


In [ ]:
# Save processed table
processed = doses[['grid_id', 'uva_hours', 'uvb_hours', 'p_uva_mw_cm2', 'p_uvb_mw_cm2', 'uva_dose_mJ_cm2', 'uvb_dose_mJ_cm2', 'mdv_uva_percent_calc', 'mdv_uvb_percent_calc']].copy()
processed = processed.rename(columns={
    'mdv_uva_percent_calc': 'mdv_uva_percent',
    'mdv_uvb_percent_calc': 'mdv_uvb_percent',
})
processed.to_csv(OUTPUT_PATH, index=False)
print(f'Written {OUTPUT_PATH.relative_to(PROJECT_ROOT)}')
processed.head()


In [ ]:
# Visualise dose gradients
fig, ax = plt.subplots(figsize=(8, 4))
sc = ax.scatter(processed['p_uva_mw_cm2'], processed['p_uvb_mw_cm2'], c=processed['mdv_uvb_percent'], cmap='viridis', s=120)
ax.set_xlabel('P (UVA) mW/cm²')
ax.set_ylabel('P (UVB) mW/cm²')
ax.set_title('%MDV UVB vs lamp power')
for _, row in processed.iterrows():
    ax.text(row['p_uva_mw_cm2'] + 0.02, row['p_uvb_mw_cm2'] + 0.005, str(int(row['grid_id'])))
fig.colorbar(sc, ax=ax, label='%MDV UVB')
plt.show()


## Summary

- Calculated doses align with the chamber schedule `%MDV` values within floating-point tolerance.
- Processed table saved to `data-processed/supplements/uv_dose_summary.csv`.
- Raw calibration files remain under `data/raw/supplements/July_26_2025_DOSE_SCHEDULE_UPDATED/Calibration_files/` for deeper audits.